## Natural Language Processing

In this exercise we will attempt to classify text messages as "SPAM" or "HAM" using TF-IDF Vectorization. Once we successfully classify our texts we will examine our results to see which words are most important to each class of text messages. 

Complete the functions below and answer the question(s) at the end. 

In [2]:
# import necessary libraries 
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import string
from nltk.corpus import stopwords
from nltk import word_tokenize

In [3]:
# read in data
df_messages = pd.read_csv('data/spam.csv', usecols=[0,1])

# convert string labels to 1 or 0 
le = LabelEncoder()
df_messages['target'] = le.fit_transform(df_messages['v1'])

# examine our data
df_messages.head()

,v1,v2,target
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


### TF-IDF

In [4]:
# separate features and labels 
X = df_messages['v2']
y = df_messages['target']

# generate a list of stopwords for TfidfVectorizer to ignore
stopwords_list = stopwords.words('english') + list(string.punctuation)

#stopwords_list[:100]

<b>1) Let's create a function that takes in our various texts along with their respective labels and uses TF-IDF to vectorize the texts.  Recall that TF-IDF helps us "vectorize" text (turn text into numbers) so we can do "math" with it.  It is used to reflect how relevant a term is in a given document in a numerical way. </b>

In [5]:
# generate tf-idf vectorization (use sklearn's TfidfVectorizer) for our data
import sklearn

def tfidf(X, y,  stopwords_list): 
    '''
    Generate train and test TF-IDF vectorization for our data set
    
    Parameters
    ----------
    X: pandas.Series object
        Pandas series of text documents to classify 
    y : pandas.Series object
        Pandas series containing label for each document
    stopwords_list: list ojbect
        List containing words and punctuation to remove. 
    Returns
    --------
    tf_idf_train :  sparse matrix, [n_train_samples, n_features]
        Vector representation of train data
    tf_idf_test :  sparse matrix, [n_test_samples, n_features]
        Vector representation of test data
    y_train : array-like object
        labels for training data
    y_test : array-like object
        labels for testing data
    vectorizer : vectorizer object
        fit TF-IDF vecotrizer object

    '''
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
    
    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    return X_train, X_test, y_train, y_test, vectorizer

In [6]:
tf_idf_train, tf_idf_test, y_train, y_test, vectorizer = tfidf(X, y, stopwords_list)

tf_idf_train

<4179x7398 sparse matrix of type '<class 'numpy.float64'>'
	with 55050 stored elements in Compressed Sparse Row format>

### Classification

<b>2) Now that we have a set of vectorized training data we can use this data to train a classifier to learn how to classify a specific text based on the vectorized version of the text. Below we have initialized a simple Naive Bayes Classifier and Random Forest Classifier. Complete the function below which will accept a classifier object, a vectorized training set, vectorized test set, and list of training labels and return a list of predictions for our training set and a separate list of predictions for our test set.</b> 

In [7]:
nb_classifier = MultinomialNB()
rf_classifier = RandomForestClassifier(n_estimators=100)

In [8]:
# create a function that takes in a classifier, trains it on our tf-idf vectors,
# and generates train and test predictiions
def classify_text(classifier, tf_idf_train, tf_idf_test, y_train):
    '''
    Train a classifier to identify whether a message is spam or ham
    
    Parameters
    ----------
    classifier: sklearn classifier
       initialized sklearn classifier (MultinomialNB, RandomForestClassifier, etc.)
    tf_idf_train : sparse matrix, [n_train_samples, n_features]
        TF-IDF vectorization of train data
    tf_idf_test : sparse matrix, [n_test_samples, n_features]
        TF-IDF vectorization of test data
    y_train : pandas.Series object
        Pandas series containing label for each document in the train set
    Returns
    --------
    train_preds :  list object
        Predictions for train data
    test_preds :  list object
        Predictions for test data
    '''
    # your code here
    # a) fit the classifier with our training data
    classifier.fit(tf_idf_train, y_train)
    
    # b) predict the labels of our train data and store them in train_preds
    train_preds = classifier.predict(tf_idf_train)
    
    # c) predict the labels of our test data and store them in test_preds
    test_preds = classifier.predict(tf_idf_test)    
    
    # d) return train_preds and test_preds
    return train_preds, test_preds

In [9]:
# generate predictions with Naive Bayes Classifier
nb_train_preds, nb_test_preds = classify_text(nb_classifier, tf_idf_train, tf_idf_test, y_train)

# evaluate performance of Naive Bayes Classifier
print(confusion_matrix(y_test, nb_test_preds))
print(accuracy_score(y_test, nb_test_preds))

[[1202    0]
 [  56  135]]
0.9597989949748744


In [10]:
# generate predictions with Random Forest Classifier
rf_train_preds, rf_test_preds = classify_text(rf_classifier, tf_idf_train, tf_idf_test, y_train)

# evaluate performance of Random Forest Classifier
print(confusion_matrix(y_test, rf_test_preds))
print(accuracy_score(y_test, rf_test_preds))

[[1202    0]
 [  34  157]]
0.9755922469490309


You can see both classifiers do a pretty good job classifying texts as either "SPAM" or "HAM". Let's figure out which words are the most important to each class of texts! Recall that Inverse Document Frequency can help us determine which words are most important in an entire corpus or group of documents. 

<b>3) Create a function that calculates the inverse document frequency (IDF) of each word in our collection of texts.</b>

In [108]:
def get_idf(class_, df, stopwords_list):
    '''
    Get ten words with lowest IDF values representing 10 most important
    words for a defined class (spam or ham)
    
    Parameters
    ----------
    class_ : str object
        string defining class 'spam' or 'ham'
    df : pandas DataFrame object
        data frame containing texts and labels
    stopwords_list: list object
        List containing words and punctuation to remove. 
    --------
    important_10 : pandas dataframe object
        Dataframe containing 10 words and respective IDF values
        representing the 10 most important words found in the texts
        associated with the defined class
    '''
    # your code here
       
    # a) generate series containing all texts associated with the defined class
   # b) initialize dictionary to count document frequency 
    # (number of documents that contain a certain word)
    # c) loop over each text and split each text into a list of its unique words 
        # d) loop over each word and if it is not in the stopwords_list add the word 
        #    to class_dict with a value of 1. if it is already in the dictionary
        #    increment it by 1                  
    # e) take our dictionary and calculate the 
    #    IDF (number of docs / number of docs containing each word) 
    #    for each word
    # f) return the 10 words with the lowest IDF 

    import math
    words = {}
    class_dict = {}
    docs = df_messages[df_messages['v1'] == 'spam'].v2
  
    for doc in docs:
        words = set(doc.split()).union(words_b)

    for word in words:
        if not word in stopwords_list:
            class_dict[word]=+1
    print(class_dict)
    N = len(docs)
    print(N)
    for word, val in class_dict.items():
        class_dict[word] = math.log(N / val)


        
    return list(class_dict.items())[0:-10]

In [109]:
get_idf('spam', df_messages, stopwords_list)

{'time': 1, 'This': 1, 'tried': 1, 'Pound': 1, 'per': 1, 'contact': 1, '10p': 1, 'u.': 1, '�750': 1, 'claim': 1, '2': 1, '087187272008': 1, 'easy,': 1, 'call': 1, 'NOW1!': 1, 'Only': 1, 'U': 1, 'prize.': 1, '2nd': 1, 'minute.': 1, 'BT-national-rate.': 1}
747


[('time', 6.616065185132817),
 ('This', 6.616065185132817),
 ('tried', 6.616065185132817),
 ('Pound', 6.616065185132817),
 ('per', 6.616065185132817),
 ('contact', 6.616065185132817),
 ('10p', 6.616065185132817),
 ('u.', 6.616065185132817),
 ('�750', 6.616065185132817),
 ('claim', 6.616065185132817),
 ('2', 6.616065185132817)]

In [101]:
get_idf('ham', df_messages, stopwords_list)

747


[6.616065185132817,
 6.616065185132817,
 6.616065185132817,
 6.616065185132817,
 6.616065185132817,
 6.616065185132817,
 6.616065185132817,
 6.616065185132817,
 6.616065185132817,
 6.616065185132817,
 6.616065185132817]

### Explain
<b> 4) Imagine that the word "school" has the highest TF-IDF value in the second document of our test data. What does that tell us about the word school? </b>

In [ ]:
# Your answer here
# This means that school is the rarest token in the second document.